In [2184]:
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.svm import SVC
from collections import Counter
from sklearn.metrics import accuracy_score

In [749]:
# read in interest rate csv file
interest = pd.read_csv("Clean_Data/clean_fedfunds.csv")
# change data types from int to string
interest["Month"] = interest["Month"].astype(str)
interest["Year"] = interest["Year"].astype(str)
# filter from 2002
interest = interest.loc[interest["Year"] >= "2002"]
interest.head()

,Year,Month,FEDFUNDS
570,2002,1,1.73
571,2002,2,1.74
572,2002,3,1.73
573,2002,4,1.75
574,2002,5,1.75


In [750]:
# read in industrial output csv file
indus = pd.read_csv("Clean_Data/Industrial.csv")
#change data types from int to string
indus["Month"] = indus["Month"].astype(str)
indus["Year"] = indus["Year"].astype(str)
indus.head()

,DATE,INDPRO,Month,Year
0,2002-01-01,88.6706,1,2002
1,2002-02-01,88.6723,2,2002
2,2002-03-01,89.3624,3,2002
3,2002-04-01,89.7933,4,2002
4,2002-05-01,90.1629,5,2002


In [751]:
# read in Retail Sales data 
sales = pd.read_csv("Clean_Data/Retail_Sales.csv")
# change out int to string
sales["Month"] = sales["Month"].astype(str)
sales["Year"] = sales["Year"].astype(str)
sales.head()

,DATE,RSXFS,Year,Month
0,2002-01-01,256307,2002,1
1,2002-02-01,257670,2002,2
2,2002-03-01,257059,2002,3
3,2002-04-01,261333,2002,4
4,2002-05-01,257573,2002,5


In [752]:
# read in inflation data
inflation = pd.read_csv("Clean_Data/inflation.csv")
#change to string
inflation["Year"] = inflation["Year"].astype(str)
inflation.head()

,Year,1,2,3,4,5,6,7,8,9,10,11,12
0,2002,2.6,2.6,2.4,2.5,2.5,2.3,2.2,2.4,2.2,2.2,2.0,1.9
1,2003,1.9,1.7,1.7,1.5,1.6,1.5,1.5,1.3,1.2,1.3,1.1,1.1
2,2004,1.1,1.2,1.6,1.8,1.7,1.9,1.8,1.7,2.0,2.0,2.2,2.2
3,2005,2.3,2.4,2.3,2.2,2.2,2.0,2.1,2.1,2.0,2.1,2.1,2.2
4,2006,2.1,2.1,2.1,2.3,2.4,2.6,2.7,2.8,2.9,2.7,2.6,2.6


In [753]:
# read in unemployment data
unemployment = pd.read_csv("Clean_Data/Unemployment.csv")
#change int to string
unemployment["Year"] = unemployment["Year"].astype(str)
unemployment.head()

,Year,1,2,3,4,5,6,7,8,9,10,11,12
0,2002,5.7,5.7,5.7,5.9,5.8,5.8,5.8,5.7,5.7,5.7,5.9,6.0
1,2003,5.8,5.9,5.9,6.0,6.1,6.3,6.2,6.1,6.1,6.0,5.8,5.7
2,2004,5.7,5.6,5.8,5.6,5.6,5.6,5.5,5.4,5.4,5.5,5.4,5.4
3,2005,5.3,5.4,5.2,5.2,5.1,5.0,5.0,4.9,5.0,5.0,5.0,4.9
4,2006,4.7,4.8,4.7,4.7,4.6,4.6,4.7,4.7,4.5,4.4,4.5,4.4


In [2281]:
# create empty dataframe
data = pd.DataFrame()
ticker_list = ["AAPL", "NVDA", "AMZN", "TSLA", "MSFT", "GOOG", "META", "GOOGL", "PEP", "COST", "AVGO", "ADBE", "CSCO", "CMCSA", "TMUS", "INTC", "TXN", "AMD", "QCOM", "AMGN", "HON", "INTU", "FISV", "MRNA", "PANW", "KDP", "KLAC", "MNST", "SNPS", "AEP", "FTNT", "KHC", "MAR", "PAYX", "ASML", "CDNS", "EXC", "ORLY", "ADSK"]
#read in tickers
for i in ticker_list:
    temp = pd.read_csv(f"Extracting_Data/{i}/{i}_monthly_1_False.csv")
    # add ticker name to each row
    temp["Ticker"] = i
    # combines each ticker csv on top of eachother
    data = pd.concat([data, temp])
data.head()

,Datetime,Open,Close,High,Low,Volume,Gain%,Ticker
0,2002-06-30 22:00:00,0.316250,0.272500,0.335536,0.246429,8118415200,-13.833992,AAPL
1,2002-07-31 22:00:00,0.269821,0.263393,0.290179,0.249464,5151686400,-2.382529,AAPL
2,2002-08-31 22:00:00,0.258750,0.258929,0.271250,0.250893,5291837600,0.069014,AAPL
3,2002-09-30 22:00:00,0.260536,0.286964,0.293571,0.238571,6611432800,10.143926,AAPL
4,2002-10-31 22:00:00,0.284643,0.276786,0.310357,0.268036,4531721600,-2.760345,AAPL


In [2282]:


#add a future column where shows the gain and add column to see where tickers overlap
data['future'] = data["Gain%"].shift(-1)
data["Mov_Ticker"] = data["Ticker"].shift(-1)
data = data.dropna()

#drop rows where ticker info overlaps
data = data.loc[data["Mov_Ticker"] == data["Ticker"]]
data.head()

,Datetime,Open,Close,High,Low,Volume,Gain%,Ticker,future,Mov_Ticker
0,2002-06-30 22:00:00,0.316250,0.272500,0.335536,0.246429,8118415200,-13.833992,AAPL,-2.382529,AAPL
1,2002-07-31 22:00:00,0.269821,0.263393,0.290179,0.249464,5151686400,-2.382529,AAPL,0.069014,AAPL
2,2002-08-31 22:00:00,0.258750,0.258929,0.271250,0.250893,5291837600,0.069014,AAPL,10.143926,AAPL
3,2002-09-30 22:00:00,0.260536,0.286964,0.293571,0.238571,6611432800,10.143926,AAPL,-2.760345,AAPL
4,2002-10-31 22:00:00,0.284643,0.276786,0.310357,0.268036,4531721600,-2.760345,AAPL,-9.874217,AAPL


In [2283]:
R_g = []
for x in data["Gain%"]:
    if x <= 0:
        R_g.append(0)
    if x > 0:
        R_g.append(1)
    #add column for calculation
data["Current_R/G"] = R_g

In [2284]:
def data_transform(data):
    #create arrays
    inflation_stat = []
    unemployment_stat = []
    #iterate through each datetime in the dataset
    for i in data["Datetime"]:
        #calculates if the month is formatted as 0# and assigns month to number #
        if i[5] == "0":
            month = i[6]
        # if formatted as ## then month assigned this number ##
        else:
            month = i[5:7]

        # locates in inflation/unemployment data when the year is the same as the current itteration
        inf_month = inflation.loc[inflation["Year"] == [i][0][:4]]
        unemp_month = unemployment.loc[unemployment["Year"] == [i][0][:4]]
        
        #locates the month of the filtered data and extracts the value
        inflation_stat.append(inf_month[month].values[0])
        unemployment_stat.append(unemp_month[month].values[0])

    # adds arrays to dataset
    data["Inflation"] = inflation_stat
    data["Unemployment"] = unemployment_stat
    # eliminates month from data
    data = data.loc[data["Datetime"] < "2022-06"]
    
    # create arrays
    industrial_stat = []
    interest_stat = []
    sales_stat = []
    #iterate through each datetime in the dataset
    for i in data["Datetime"]:
        #calculates if the month is formatted as 0# and assigns month to number #
        if i[5] == "0":
            month = i[6]
        # if formatted as ## then month assigned this number ##
        else:
            month = i[5:6]
        
        # filters datasets with the year in the iteration and when the month of the dataset is = "month"
        row = interest.loc[(interest["Year"] == [i][0][:4]) & (interest["Month"] == month)]
        ind = indus.loc[(indus["Year"] == [i][0][:4]) & (indus["Month"] == month)]
        sal = sales.loc[(sales["Year"] == [i][0][:4]) & (sales["Month"] == month)]
        
        # append stats to arrays
        interest_stat.append(row["FEDFUNDS"].values[0])
        industrial_stat.append(ind["INDPRO"].values[0])
        sales_stat.append(sal["RSXFS"].values[0])
    # adds arrays to dataset
    data["Industrial"] = industrial_stat
    data["Interest"] = interest_stat
    data["Retail_Sales"] = sales_stat
    
    # add new columns with percent changed
    data["Interest_pct"] = data["Interest"].pct_change()
    data["Inflation_pct"] = data["Inflation"].pct_change()
    data["Unemployment_pct"] = data["Unemployment"].pct_change()
    data["Industrial_pct"] = data["Industrial"].pct_change()
    data["Volume_pct"] = data["Volume"].pct_change()
    data["Retail_Sales_pct"] = data["Retail_Sales"].pct_change() 
    data = data.dropna()

    # calculates if the future gain is positive or negative
    R_g = []
    for x in data["future"]:
        if x <= 0:
            R_g.append(0)
        if x > 0:
            R_g.append(1)
    #add column for calculation
    data["R/G"] = R_g
    
    return data


In [2285]:
# call function
data = data_transform(data)
data.head()

,Datetime,Open,Close,High,Low,Volume,Gain%,Ticker,future,Mov_Ticker,...,Industrial,Interest,Retail_Sales,Interest_pct,Inflation_pct,Unemployment_pct,Industrial_pct,Volume_pct,Retail_Sales_pct,R/G
1,2002-07-31 22:00:00,0.269821,0.263393,0.290179,0.249464,5151686400,-2.382529,AAPL,0.069014,AAPL,...,90.8761,1.73,262769,-0.011429,-0.043478,0.000000,-0.000349,-0.365432,0.011483,1
2,2002-08-31 22:00:00,0.258750,0.258929,0.271250,0.250893,5291837600,0.069014,AAPL,10.143926,AAPL,...,90.7850,1.74,265043,0.005780,0.090909,-0.017241,-0.001002,0.027205,0.008654,1
3,2002-09-30 22:00:00,0.260536,0.286964,0.293571,0.238571,6611432800,10.143926,AAPL,-2.760345,AAPL,...,90.8931,1.75,260626,0.005747,-0.083333,0.000000,0.001191,0.249364,-0.016665,0
4,2002-10-31 22:00:00,0.284643,0.276786,0.310357,0.268036,4531721600,-2.760345,AAPL,-9.874217,AAPL,...,88.6706,1.73,256307,-0.011429,0.000000,0.000000,-0.024452,-0.314563,-0.016572,0
5,2002-11-30 22:00:00,0.283929,0.255893,0.287500,0.246071,4530439200,-9.874217,AAPL,0.000000,AAPL,...,88.6706,1.73,256307,0.000000,-0.090909,0.035088,0.000000,-0.000283,0.000000,0


In [2414]:
#index to test/predict for
tester = pd.read_csv("Extracting_Data/QQQ/QQQ_monthly_1_False.csv")
#add a future column where shows the gain for next row
tester['future'] = tester["Gain%"].shift(-1)
tester = tester.dropna()
tester.head()

,Datetime,Open,Close,High,Low,Volume,Gain%,future
0,2002-06-30 22:00:00,25.97,23.85,26.55,21.64,2668049100,-8.163265,-1.053075
1,2002-07-31 22:00:00,23.74,23.49,26.21,21.30,2043810216,-1.053075,-10.147441
2,2002-08-31 22:00:00,23.06,20.72,24.35,20.49,1669047392,-10.147441,17.407939
3,2002-09-30 22:00:00,20.91,24.55,25.04,19.76,2129150120,17.407939,13.746410
4,2002-10-31 22:00:00,24.37,27.72,28.29,24.14,1669954692,13.746410,-14.250528


In [2415]:
R_g = []
for x in tester["Gain%"]:
    if x <= 0:
        R_g.append(0)
    if x > 0:
        R_g.append(1)
    #add column for calculation
tester["Current_R/G"] = R_g

In [2416]:
def data_transform_test(data):
    #create arrays
    inflation_stat = []
    unemployment_stat = []
    #iterate through each datetime in the dataset
    for i in data["Datetime"]:
        #calculates if the month is formatted as 0# and assigns month to number #
        if i[5] == "0":
            month = i[6]
        # if formatted as ## then month assigned this number ##
        else:
            month = i[5:7]

        # locates in inflation/unemployment data when the year is the same as the current itteration
        inf_month = inflation.loc[inflation["Year"] == [i][0][:4]]
        unemp_month = unemployment.loc[unemployment["Year"] == [i][0][:4]]
        
        #locates the month of the filtered data and extracts the value
        inflation_stat.append(inf_month[month].values[0])
        unemployment_stat.append(unemp_month[month].values[0])

    # adds arrays to dataset
    data["Inflation"] = inflation_stat
    data["Unemployment"] = unemployment_stat
    # eliminates month from data
    data = data.loc[data["Datetime"] < "2022-06"]
    
    # create arrays
    industrial_stat = []
    interest_stat = []
    sales_stat = []
    #iterate through each datetime in the dataset
    for i in data["Datetime"]:
        #calculates if the month is formatted as 0# and assigns month to number #
        if i[5] == "0":
            month = i[6]
        # if formatted as ## then month assigned this number ##
        else:
            month = i[5:6]
        
        # filters datasets with the year in the iteration and when the month of the dataset is = "month"
        row = interest.loc[(interest["Year"] == [i][0][:4]) & (interest["Month"] == month)]
        ind = indus.loc[(indus["Year"] == [i][0][:4]) & (indus["Month"] == month)]
        sal = sales.loc[(sales["Year"] == [i][0][:4]) & (sales["Month"] == month)]
        
        # append stats to arrays
        interest_stat.append(row["FEDFUNDS"].values[0])
        industrial_stat.append(ind["INDPRO"].values[0])
        sales_stat.append(sal["RSXFS"].values[0])
    # adds arrays to dataset
    data["Industrial"] = industrial_stat
    data["Interest"] = interest_stat
    data["Retail_Sales"] = sales_stat
    
    # add new columns with percent changed
    data["Interest_pct"] = data["Interest"].pct_change()
    data["Inflation_pct"] = data["Inflation"].pct_change()
    data["Unemployment_pct"] = data["Unemployment"].pct_change()
    data["Industrial_pct"] = data["Industrial"].pct_change()
    data["Volume_pct"] = data["Volume"].pct_change()
    data["Retail_Sales_pct"] = data["Retail_Sales"].pct_change() 
    data = data.dropna()

    # calculates if the future gain is positive or negative
    R_g = []
    for x in data["future"]:
        if x <= 0:
            R_g.append(0)
        if x > 0:
            R_g.append(1)
    #add column for calculation
    data["R/G"] = R_g
    return data

In [2417]:
tester = data_transform_test(tester)
tester.tail()

,Datetime,Open,Close,High,Low,Volume,Gain%,future,Current_R/G,Inflation,...,Industrial,Interest,Retail_Sales,Interest_pct,Inflation_pct,Unemployment_pct,Industrial_pct,Volume_pct,Retail_Sales_pct,R/G
234,2021-12-31 22:00:00,399.05,363.05,402.28,334.15,1848326612,-9.021426,-4.837692,0,5.5,...,99.4076,0.09,513269,0.000000,0.122449,-0.071429,0.000000,0.497130,0.000000,0
235,2022-01-31 22:00:00,364.43,346.80,370.10,318.26,1523185568,-4.837692,4.856110,0,6.0,...,102.5479,0.08,576000,-0.111111,0.090909,0.025641,0.031590,-0.175911,0.122219,1
236,2022-02-28 22:00:00,345.75,362.54,371.83,317.45,1686866041,4.856110,-13.660042,1,6.4,...,103.5578,0.08,581768,0.000000,0.066667,-0.050000,0.009848,0.107459,0.010014,0
237,2022-03-31 22:00:00,362.81,313.25,369.31,312.60,1501958749,-13.660042,-1.454464,0,6.5,...,104.0733,0.20,587570,1.500000,0.015625,-0.052632,0.004978,-0.109616,0.009973,0
238,2022-04-30 22:00:00,312.83,308.28,330.29,280.21,1951162721,-1.454464,-8.252005,0,6.2,...,105.4868,0.33,590259,0.650000,-0.046154,0.000000,0.013582,0.299079,0.004576,0


In [2402]:
data.columns

Index(['Datetime', 'Open', 'Close', 'High', 'Low', 'Volume', 'Gain%', 'Ticker',
       'future', 'Mov_Ticker', 'Current_R/G', 'Inflation', 'Unemployment',
       'Industrial', 'Interest', 'Retail_Sales', 'Interest_pct',
       'Inflation_pct', 'Unemployment_pct', 'Industrial_pct', 'Volume_pct',
       'Retail_Sales_pct', 'R/G'],
      dtype='object')

In [2454]:
#columns to train model for stock tickers
X = testerme[[ "Interest_pct","Unemployment_pct", "Industrial_pct", "Retail_Sales_pct","Inflation_pct", "Inflation", "Interest", "Industrial", "Retail_Sales", "Unemployment", "Current_R/G", "Volume_pct"]]
y = testerme["R/G"]

In [2455]:
#columns from index to test
TestX = data1[[ "Interest_pct", "Unemployment_pct", "Industrial_pct", "Retail_Sales_pct", "Inflation_pct","Inflation", "Interest", "Industrial", "Retail_Sales", "Unemployment", "Current_R/G", "Volume_pct"]]
Testy = data1["R/G"]

In [2470]:
#train test and split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
# Standardize data set
scaler = StandardScaler()
# Fit X_train data
scaler.fit(X_train)
#Transform scaled data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


# implement random oversampling
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)

# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128)

# Fitting the model
rf_model = rf_model.fit(X_resampled, y_resampled)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)

accuracy_score(y_test, y_pred)

0.6282548476454294

In [2471]:
scaled = scaler.fit_transform(TestX)
# Evaluate the model
y_pred = rf_model.predict(scaled)

accuracy_score(Testy, y_pred)

0.7333333333333333

In [2464]:
pd.DataFrame({"Prediction": y_pred, "Actual": Testy})

,Prediction,Actual
234,0,0
8,0,1
70,1,0
206,1,1
100,1,1
139,0,0
109,0,0
28,1,1
113,1,1
82,1,1


In [2465]:
for i in range(50):
    #train test and split
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    # Standardize data set
    scaler = StandardScaler()
    # Fit X_train data
    scaler.fit(X_train)
    #Transform scaled data
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # implement random oversampling
    from imblearn.over_sampling import RandomOverSampler
    ros = RandomOverSampler(random_state=1)
    X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)

    # Create a random forest classifier.
    rf_model = RandomForestClassifier(n_estimators=128)

    # Fitting the model
    rf_model = rf_model.fit(X_resampled, y_resampled)

    scaled = scaler.fit_transform(TestX)
    # Evaluate the model
    y_pred = rf_model.predict(scaled)

    accuracy_score(Testy, y_pred)

    print(accuracy_score(Testy, y_pred))

0.5
0.5
0.6
0.6666666666666666
0.5666666666666667
0.6333333333333333
0.6333333333333333
0.6
0.5666666666666667
0.6666666666666666
0.5333333333333333
0.6
0.5666666666666667
0.6
0.6666666666666666
0.5666666666666667
0.6
0.6333333333333333


KeyboardInterrupt: 

In [2472]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(Testy, y_pred)

array([[ 5,  7],
       [ 1, 17]])

In [2473]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(Testy, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.83      0.42      0.94      0.56      0.63      0.37        12
          1       0.71      0.94      0.42      0.81      0.63      0.41        18

avg / total       0.76      0.73      0.63      0.71      0.63      0.40        30



In [2453]:

# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.1496280878897855, 'Current_R/G'),
 (0.10065609904896479, 'Unemployment_pct'),
 (0.09953634321323387, 'Industrial_pct'),
 (0.0983074906452893, 'Retail_Sales'),
 (0.09038276657997922, 'Industrial'),
 (0.0885646666907374, 'Unemployment'),
 (0.08570738637994733, 'Interest_pct'),
 (0.0751567210658804, 'Interest'),
 (0.07329595794050822, 'Inflation'),
 (0.07230244217390439, 'Retail_Sales_pct'),
 (0.06646203837176969, 'Inflation_pct')]

In [2418]:
data1 = tester.sample(n=30)
data1.head()

,Datetime,Open,Close,High,Low,Volume,Gain%,future,Current_R/G,Inflation,...,Industrial,Interest,Retail_Sales,Interest_pct,Inflation_pct,Unemployment_pct,Industrial_pct,Volume_pct,Retail_Sales_pct,R/G
234,2021-12-31 22:00:00,399.05,363.050,402.2800,334.15,1848326612,-9.021426,-4.837692,0,5.5,...,99.4076,0.09,513269,0.000000,0.122449,-0.071429,0.000000,0.497130,0.000000,0
8,2003-02-28 22:00:00,25.27,25.250,27.3800,23.54,1670716196,-0.079145,7.900943,0,1.7,...,91.4940,1.26,263188,0.016129,-0.105263,0.017241,0.001191,0.308700,-0.015126,1
70,2008-04-30 22:00:00,47.24,50.010,50.4700,47.22,2682987554,5.863675,-9.515224,1,2.3,...,100.9098,2.28,334843,-0.126437,-0.041667,-0.019608,-0.006699,0.024956,-0.001044,0
206,2019-08-31 22:00:00,186.26,188.810,194.7100,185.03,556686903,1.369054,3.978052,1,2.4,...,102.6882,2.13,455967,-0.112500,0.090909,0.000000,0.006187,-0.319051,0.005070,1
100,2010-10-31 22:00:00,52.38,52.085,54.0401,50.85,1565231519,-0.563192,3.007377,0,0.6,...,89.2744,0.11,308343,-0.421053,-0.250000,-0.010526,-0.039288,0.017397,-0.037213,1


In [2419]:
p = data1["Datetime"].tolist()

In [2420]:
testerme = data.copy()

In [2421]:
for fuck in p:
    testerme = testerme.loc[testerme["Datetime"] != fuck]

In [1955]:
len(data)

5899

In [2474]:
date = []
for i in data["Datetime"]:
    date.append(i[0:10])
data["Datetime"] = date    

In [2475]:
data.to_csv("Tech_Stocks.csv")